In [1]:
import sys
sys.path.append('..')

from data_parsers import databento_file_parser
from executor import SimpleExecutor1

import pandas as pd
import math
import numpy as np

from tqdm import tqdm
import math
import numpy
import matplotlib.pyplot as plt
import random
import numpy as np

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 80)

In [4]:
data1 = pd.read_csv('outputs/backtest_output_avellaneda.csv')
data1['strategy'] = 'Avellaneda'

data2 = pd.read_csv('outputs/backtest_output_ho_and_stoll.csv')
data2['strategy'] = 'Ho_Stoll'

data_df = pd.concat([data1, data2])
data_df

,Unnamed: 0,Unnamed: 1,eod_position,eod_cash,net_pnl,trading_pnl,avg_size,avg_size_square,trading_volume,volatility,net_return,utility,strategy
0,MSFT,2023-05-15 14:00:00,0,12.84,27.06,31.120,4.996581,66.078342,1422,0.064561,1.155,6.233419,Avellaneda
1,MSFT,2023-05-15 14:30:00,26,-8006.99,44.07,35.320,5.031858,57.212357,1238,0.065185,-1.565,9.219538,Avellaneda
2,MSFT,2023-05-15 15:00:00,-8,2543.50,76.02,20.245,8.567006,173.583203,988,0.050350,-0.445,11.616819,Avellaneda
3,MSFT,2023-05-15 15:30:00,9,-2768.40,27.02,23.295,4.524061,39.940398,767,0.045363,-0.190,9.545058,Avellaneda
4,MSFT,2023-05-15 16:00:00,5,-1472.28,80.95,29.705,10.799980,185.130634,763,0.034280,0.705,18.153171,Avellaneda
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,MSFT,2023-05-19 17:30:00,25,-7912.63,42.32,44.145,10.000786,195.225920,1745,0.031336,0.590,7.372936,Ho_Stoll
56,MSFT,2023-05-19 18:00:00,5,-1636.74,-26.99,48.085,7.545683,127.656517,1985,0.040733,-0.520,3.420828,Ho_Stoll
57,MSFT,2023-05-19 18:30:00,11,-3495.42,13.76,18.975,6.180239,93.864848,1063,0.034842,-0.015,4.776233,Ho_Stoll
58,MSFT,2023-05-19 19:00:00,7,-2224.29,14.91,21.975,7.026637,106.719797,1271,0.032988,-0.075,4.547034,Ho_Stoll


In [10]:
columns = ['net_pnl', 'trading_pnl', 'avg_size', 'trading_volume', 'utility']

print(data_df.groupby('strategy')[columns].mean())
print(data_df.groupby('strategy')[columns].std())

              net_pnl  trading_pnl  avg_size  trading_volume   utility
strategy                                                              
Avellaneda  15.139333    36.797833  6.848771     1255.050000  6.617056
Ho_Stoll    17.112500    56.766250  7.107574     2265.883333  5.469845
              net_pnl  trading_pnl  avg_size  trading_volume   utility
strategy                                                              
Avellaneda  31.626403    32.455081  2.877351      938.528065  3.636426
Ho_Stoll    28.975480    42.013307  1.522605     1308.447289  1.712752


In [13]:
data_df['opening_hours'] = pd.to_datetime(data_df['Unnamed: 1']).dt.hour <= 14
data_df.groupby(['strategy', 'opening_hours'])[columns].mean()

net_pnl  trading_pnl  avg_size  trading_volume  \
strategy   opening_hours                                                   
Avellaneda False          15.6276      36.8536  7.034568         1210.12   
           True           12.6980      36.5190  5.919785         1479.70   
Ho_Stoll   False          17.6842      56.4478  7.246704         2195.76   
           True           14.2540      58.3585  6.411925         2616.50   

                           utility  
strategy   opening_hours            
Avellaneda False          6.803706  
           True           5.683805  
Ho_Stoll   False          5.587838  
           True           4.879877

In [14]:
data_df.groupby(['strategy', 'opening_hours'])[columns].std()

net_pnl  trading_pnl  avg_size  trading_volume  \
strategy   opening_hours                                                     
Avellaneda False          31.750462    34.839986  3.086653      986.586811   
           True           32.567038    17.218421  1.119020      638.516171   
Ho_Stoll   False          29.704812    44.848938  1.598278     1359.499886   
           True           26.248089    24.836361  0.803051      998.392180   

                           utility  
strategy   opening_hours            
Avellaneda False          3.816700  
           True           2.493273  
Ho_Stoll   False          1.798665  
           True           1.073735

In [15]:
data_df['Volatility'] = np.where(data_df['volatility'] <= data_df['volatility'].median(), 'Low', 'High')

data_df.groupby(['strategy', 'Volatility'])[columns].mean()

net_pnl  trading_pnl  avg_size  trading_volume  \
strategy   Volatility                                                     
Avellaneda High        16.421000    53.581000  5.874807     1809.833333   
           Low         13.857667    20.014667  7.822735      700.266667   
Ho_Stoll   High        20.233333    80.670833  6.678385     3055.033333   
           Low         13.991667    32.861667  7.536764     1476.733333   

                        utility  
strategy   Volatility            
Avellaneda High        6.550069  
           Low         6.684044  
Ho_Stoll   High        5.738748  
           Low         5.200941

In [16]:
data_df.groupby(['strategy', 'Volatility'])[columns].std()

net_pnl  trading_pnl  avg_size  trading_volume  \
strategy   Volatility                                                     
Avellaneda High        36.269670    36.539460  1.279579      986.244674   
           Low         26.759330    15.002636  3.639208      427.281296   
Ho_Stoll   High        35.962169    46.318534  0.979491     1355.918406   
           Low         19.866539    16.230478  1.837412      596.762353   

                        utility  
strategy   Volatility            
Avellaneda High        2.936658  
           Low         4.274334  
Ho_Stoll   High        1.742509  
           Low         1.668007

In [9]:
momentum = data_df['net_return'].abs()

data_df['Momentum'] = np.where(momentum <= momentum.median(), 'Low', 'High')

print(data_df.groupby(['strategy', 'Momentum'])[columns].mean())
print(data_df.groupby(['strategy', 'Momentum'])[columns].std())

                       net_pnl  trading_pnl  avg_size  trading_volume  \
strategy   Momentum                                                     
Avellaneda High      24.058000    36.411500  6.686934     1306.466667   
           Low        6.220667    37.184167  7.010608     1203.633333   
Ho_Stoll   High      21.404000    54.241167  6.961929     2293.933333   
           Low       12.821000    59.291333  7.253220     2237.833333   

                      utility  
strategy   Momentum            
Avellaneda High      7.567748  
           Low       5.666364  
Ho_Stoll   High      5.584321  
           Low       5.355368  
                       net_pnl  trading_pnl  avg_size  trading_volume  \
strategy   Momentum                                                     
Avellaneda High      33.491616    22.553086  2.000911      861.429435   
           Low       27.362201    40.423210  3.575746     1022.014255   
Ho_Stoll   High      31.442088    30.593871  1.390542     1258.063480   
    

In [17]:
data_df

,Unnamed: 0,Unnamed: 1,eod_position,eod_cash,net_pnl,trading_pnl,avg_size,avg_size_square,trading_volume,volatility,net_return,utility,strategy,opening_hours,Volatility,Momentum
0,MSFT,2023-05-15 14:00:00,0,12.84,27.06,31.120,4.996581,66.078342,1422,0.064561,1.155,6.233419,Avellaneda,True,High,High
1,MSFT,2023-05-15 14:30:00,26,-8006.99,44.07,35.320,5.031858,57.212357,1238,0.065185,-1.565,9.219538,Avellaneda,True,High,High
2,MSFT,2023-05-15 15:00:00,-8,2543.50,76.02,20.245,8.567006,173.583203,988,0.050350,-0.445,11.616819,Avellaneda,False,High,High
3,MSFT,2023-05-15 15:30:00,9,-2768.40,27.02,23.295,4.524061,39.940398,767,0.045363,-0.190,9.545058,Avellaneda,False,High,Low
4,MSFT,2023-05-15 16:00:00,5,-1472.28,80.95,29.705,10.799980,185.130634,763,0.034280,0.705,18.153171,Avellaneda,False,Low,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,MSFT,2023-05-19 17:30:00,25,-7912.63,42.32,44.145,10.000786,195.225920,1745,0.031336,0.590,7.372936,Ho_Stoll,False,Low,High
56,MSFT,2023-05-19 18:00:00,5,-1636.74,-26.99,48.085,7.545683,127.656517,1985,0.040733,-0.520,3.420828,Ho_Stoll,False,Low,High
57,MSFT,2023-05-19 18:30:00,11,-3495.42,13.76,18.975,6.180239,93.864848,1063,0.034842,-0.015,4.776233,Ho_Stoll,False,Low,Low
58,MSFT,2023-05-19 19:00:00,7,-2224.29,14.91,21.975,7.026637,106.719797,1271,0.032988,-0.075,4.547034,Ho_Stoll,False,Low,Low
